In [4]:
import torch
import torch.nn.functional as F

class Linear_fw_no_bias(torch.nn.Linear): 
    def __init__(self, in_features, out_features):
        super(Linear_fw_no_bias, self).__init__(in_features, out_features, bias=False)
        self.weight.fast = None

    def forward(self, x):
        if self.weight.fast is not None:
            out = F.linear(x, self.weight.fast)
        else:
            out = super(Linear_fw, self).forward(x)
        return out

In [2]:
from torch import nn


#class Encoder(nn.Module):
#    def __init__(self, mean, ):
#        super(Encoder, self).__init__()
#        self.mean =

In [5]:
import torch

encoder = torch.nn.Linear(1, 1, bias=False) # phi_encoder
decoder = torch.nn.Linear(1, 1, bias=False) # phi_decoder

model = Linear_fw_no_bias(1, 1) # theta

encoder.weight, decoder.weight, model.weight

(Parameter containing:
 tensor([[-0.7962]], requires_grad=True),
 Parameter containing:
 tensor([[0.9256]], requires_grad=True),
 Parameter containing:
 tensor([[0.4684]], requires_grad=True))

In [6]:
tasks_data = torch.cat((torch.rand((5, 1)), torch.tensor([[1., 0., 1., 1., 0.]]).view(5, 1)), dim=1)
tasks_data

tensor([[0.0275, 1.0000],
        [0.5516, 0.0000],
        [0.9406, 1.0000],
        [0.8959, 1.0000],
        [0.3911, 0.0000]])

In [13]:
#mu = torch.nn.Parameter()
#sigma = torch.nn.Parameter()

torch.normal(torch.tensor(0.), torch.tensor(1.)) * sigma + mu

tensor(-0.2762)

In [7]:
n_adaptation_steps = 1000
epochs = 10
lr = 0.5
inner_lr = 0.5

optimizer = torch.optim.SGD([*encoder.parameters(), *decoder.parameters()], lr)
criterion = torch.nn.BCEWithLogitsLoss()

for i in range(epochs):
    loss_all = []

    for (x, y) in tasks_data:
        x = x.unsqueeze(0)

        encoder.zero_grad()
        decoder.zero_grad()
        z = encoder(x)

        w = decoder(z)
        for weight in model.parameters():
            weight.fast = w

        for i in range(n_adaptation_steps):
            loss = criterion(torch.sigmoid(model(x)), y)
            grad = torch.autograd.grad(loss, z, create_graph=True)[0]
            z = z - inner_lr * grad
            
            w = decoder(z)
            for weight in model.parameters():
                weight.fast = w        

        loss_all.append(loss)

    loss_q = torch.stack(loss_all).sum(0)
    loss_q.backward()
    print(f"loss: {loss_q}")
    optimizer.step()

loss: 2.5153753757476807
loss: 2.512507915496826
loss: 2.510213851928711
loss: 2.5083136558532715
loss: 2.5066981315612793
loss: 2.505297899246216
loss: 2.5040643215179443
loss: 2.5029635429382324
loss: 2.5019707679748535
loss: 2.501067638397217


In [9]:
encoder.weight, decoder.weight, model.weight.fast

(Parameter containing:
 tensor([[-0.7966]], requires_grad=True),
 Parameter containing:
 tensor([[1.2002]], requires_grad=True),
 tensor([-9.9105], grad_fn=<SqueezeBackward4>))